## Procedural memory

In [1]:
%%capture cap --no-stderr
%pip install -U langmem langgraph

In [2]:
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()
store.put(("instructions",), key="agent_instructions", value={"prompt": "Write good emails."})

In [3]:
from langgraph.prebuilt import create_react_agent
from langgraph.config import get_store

def draft_email(to: str, subject: str, body: str):
    """Submit an email draft."""
    return "Draft saved succesfully."

def prompt(state):
    item = store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

agent = create_react_agent("anthropic:claude-3-5-sonnet-latest", prompt=prompt, tools=[draft_email], store=store)

In [4]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]}
)
result['messages'][1].pretty_print()

================================== Ai Message ==================================

[{'citations': None, 'text': "I'll help you draft an email to schedule the follow-up meeting.", 'type': 'text'}, {'id': 'toolu_01Reo41SH7Sx72LVzt2iCEBi', 'input': {'to': 'joe@langchain.dev', 'subject': 'Follow-up Meeting - Thursday at Noon', 'body': "Hi Joe,\n\nI'd like to schedule a follow-up meeting with you for Thursday at 12:00 PM. \n\nPlease let me know if this time works for you.\n\nBest regards"}, 'name': 'draft_email', 'type': 'tool_use'}]
Tool Calls:
  draft_email (toolu_01Reo41SH7Sx72LVzt2iCEBi)
 Call ID: toolu_01Reo41SH7Sx72LVzt2iCEBi
  Args:
    to: joe@langchain.dev
    subject: Follow-up Meeting - Thursday at Noon
    body: Hi Joe,

I'd like to schedule a follow-up meeting with you for Thursday at 12:00 PM. 

Please let me know if this time works for you.

Best regards


# Update the prompt

In [5]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer("anthropic:claude-3-5-sonnet-latest")

In [6]:
current_prompt = store.get(("instructions",), key="agent_instructions").value["prompt"]
feedback = {"request": "Always sign off from 'William'; for meeting requests, offer to schedule on Zoom or Google Meet"}

optimizer_result = optimizer.invoke({"prompt": current_prompt, "trajectories": [(result["messages"], feedback)]})

In [7]:
print(optimizer_result)

Write professional emails following these guidelines:
1. Always sign off with 'William'
2. For meeting requests:
   - Include the meeting time and date
   - Offer both Zoom and Google Meet as platform options
3. Keep the tone professional but friendly
4. Include a clear subject line

Example meeting request:
Subject: Follow-up Meeting - Thursday at Noon
Body:
Hi [Name],

I'd like to schedule a follow-up meeting with you for [Day] at [Time].

We can meet via Zoom or Google Meet - please let me know your preference.

Please let me know if this time works for you.

Best regards,
William


In [8]:
store.put(("instructions",), key="agent_instructions", value={"prompt": optimizer_result})

In [9]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]}
)
result['messages'][1].pretty_print()

================================== Ai Message ==================================

[{'citations': None, 'text': "I'll help you draft a meeting request email to Joe following the guidelines.", 'type': 'text'}, {'id': 'toolu_01BUKzWhkWje1tfyJ4BjRVdo', 'input': {'to': 'joe@langchain.dev', 'subject': 'Follow-up Meeting - Thursday at Noon', 'body': "Hi Joe,\n\nI'd like to schedule a follow-up meeting with you for Thursday at 12:00 PM.\n\nWe can meet via Zoom or Google Meet - please let me know your preference.\n\nPlease let me know if this time works for you.\n\nBest regards,\nWilliam"}, 'name': 'draft_email', 'type': 'tool_use'}]
Tool Calls:
  draft_email (toolu_01BUKzWhkWje1tfyJ4BjRVdo)
 Call ID: toolu_01BUKzWhkWje1tfyJ4BjRVdo
  Args:
    to: joe@langchain.dev
    subject: Follow-up Meeting - Thursday at Noon
    body: Hi Joe,

I'd like to schedule a follow-up meeting with you for Thursday at 12:00 PM.

We can meet via Zoom or Google Meet - please let me know your preference.

Please let m

In [10]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" : "Let roger@langchain.dev know that the release should be later by 4:00 PM."}]}
)
result['messages'][1].pretty_print()

================================== Ai Message ==================================

[{'citations': None, 'text': "I'll help you draft an email to Roger about the release timing.", 'type': 'text'}, {'id': 'toolu_01RHJf4D6T96t9VVo5xwDQGy', 'input': {'to': 'roger@langchain.dev', 'subject': 'Release Update - 4:00 PM Today', 'body': 'Hi Roger,\n\nI wanted to let you know that the release should be completed by 4:00 PM today.\n\nPlease let me know if you have any questions.\n\nBest regards,\nWilliam'}, 'name': 'draft_email', 'type': 'tool_use'}]
Tool Calls:
  draft_email (toolu_01RHJf4D6T96t9VVo5xwDQGy)
 Call ID: toolu_01RHJf4D6T96t9VVo5xwDQGy
  Args:
    to: roger@langchain.dev
    subject: Release Update - 4:00 PM Today
    body: Hi Roger,

I wanted to let you know that the release should be completed by 4:00 PM today.

Please let me know if you have any questions.

Best regards,
William


## Multi-agent

In [11]:
%%capture cap --no-stderr
%pip install -U langgraph-supervisor

In [12]:
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph.config import get_store

store = InMemoryStore()

store.put(("instructions",), key="email_agent", value={"prompt": "Write good emails. Repeat your draft content to the user after submitting."})
store.put(("instructions",), key="twitter_agent", value={"prompt": "Write fire tweets. Repeat the tweet content to the user upon submission."})

## Email agent
def draft_email(to: str, subject: str, body: str):
    """Submit an email draft."""
    return "Draft saved succesfully."

def prompt_email(state):
    item = store.get(("instructions",), key="email_agent")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

email_agent = create_react_agent(
    "anthropic:claude-3-5-sonnet-latest", 
    prompt=prompt_email, 
    tools=[draft_email], 
    store=store,
    name="email_assistant",
)

## Tweet

def tweet(to: str, subject: str, body: str):
    """Poast a tweet."""
    return "Legendary."

def prompt_social_media(state):
    item = store.get(("instructions",), key="twitter_agent")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

social_media_agent = create_react_agent(
    "anthropic:claude-3-5-sonnet-latest", 
    prompt=prompt_social_media, 
    tools=[tweet], 
    store=store,
    name="social_media_agent",
)

In [13]:
from langgraph_supervisor import create_supervisor

# Create supervisor workflow
workflow = create_supervisor(
    [email_agent, social_media_agent],
    model="anthropic:claude-3-5-sonnet-latest",
    prompt=(
        "You are a team supervisor managing email and tweet assistants to help with correspondance."
    )
)

# Compile and run
app = workflow.compile(store=store)

In [14]:
result = app.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]},
)

In [15]:
result["messages"][3].pretty_print()

================================== Ai Message ==================================
Name: email_assistant

I've drafted the email with the following content:

To: joe@langchain.dev
Subject: Follow-up Meeting - Thursday at Noon
Body:
Hi Joe,

I'd like to schedule a follow-up meeting with you for Thursday at noon. Please let me know if this time works for you.

Looking forward to our discussion.

Best regards

The email has been drafted and is ready to be sent. Let me know if you'd like any changes to the content.


In [16]:
from langmem import create_multi_prompt_optimizer

feedback = {"request": "Always sign off emails from 'William'; for meeting requests, offer to schedule on Zoom or Google Meet"}

optimizer = create_multi_prompt_optimizer("anthropic:claude-3-5-sonnet-latest")

In [17]:
from langmem import Prompt
?Prompt

Init signature: Prompt(self, /, *args, **kwargs)
Docstring:     
TypedDict for structured prompt management and optimization.

Example:
    ```python
    from langmem import Prompt

    prompt = Prompt(
        name="extract_entities",
        prompt="Extract key entities from the text:",
        update_instructions="Make minimal changes, only address where"
        " errors have occurred after reasoning over why they occur.",
        when_to_update="If there seem to be errors in recall of named entities.",
    )
    ```

The name and prompt fields are required. Optional fields control optimization:
- update_instructions: Guidelines for modifying the prompt
- when_to_update: Dependencies between prompts during optimization

Use in the prompt optimizers.
File:           ~/code/lc/community/langmem/src/langmem/prompts/types.py
Type:           _TypedDictMeta
Subclasses:     

In [18]:
email_prompt = store.get(("instructions",), key="email_agent").value['prompt']
tweet_prompt = store.get(("instructions",), key="twitter_agent").value['prompt']

email_prompt = {
    "name": "email_prompt",
    "prompt": email_prompt,
    "when_to_update": "Only if feedback is provided indicating email writing performance needs improved."
}
tweet_prompt = {
    "name": "tweet_prompt",
    "prompt": tweet_prompt,
    "when_to_update": "Only if tweet writing generation needs improvement."
}


optimizer_result = optimizer.invoke({"prompts": [tweet_prompt, email_prompt], "trajectories": [(result["messages"], feedback)]})

In [19]:
optimizer_result

[{'name': 'tweet_prompt',
  'prompt': 'Write fire tweets. Repeat the tweet content to the user upon submission.',
  'when_to_update': 'Only if tweet writing generation needs improvement.'},
 {'name': 'email_prompt',
  'prompt': "Write professional emails following these requirements:\n1. Include a clear subject line and appropriate greeting\n2. Write concise and clear content\n3. For meeting requests, always include Zoom or Google Meet as platform options\n4. Always sign off emails with 'Best regards, William'\n5. Repeat your draft content to the user after submitting",
  'when_to_update': 'Only if feedback is provided indicating email writing performance needs improved.'}]

In [20]:
store.put(("instructions",), key="email_agent", value={"prompt": optimizer_result[1]['prompt']})

In [21]:
result = app.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]},
)

In [22]:
result["messages"][3].pretty_print()

================================== Ai Message ==================================
Name: email_assistant

I've drafted the email with:

To: joe@langchain.dev
Subject: Follow-up Meeting Request - Thursday at Noon
Body:
Hi Joe,

I hope this email finds you well. I would like to schedule a follow-up meeting with you for this Thursday at 12:00 PM.

We can meet via Zoom or Google Meet, whichever you prefer. Please let me know which platform works best for you, and I'll send over the meeting details.

Looking forward to our discussion.

Best regards,
William

The email has been drafted with a clear subject line, professional greeting, meeting time details, video conferencing options, and appropriate sign-off. Would you like me to modify anything in the draft?
